# 2021 APRA data for Jersey City

In [1]:
import pandas as pd

In [2]:
# PDFs converted manually using Tabula for Mac

In [3]:
df_abandoned_2021 = pd.read_csv('./data/abandoned_2021_07.csv')
df_abandoned_2021['full_address'] = df_abandoned_2021.address + ", Jersey City, NJ"

In [4]:
df_vacant_2021 = pd.read_csv('./data/vacant_2021_07.csv')
df_vacant_2021['full_address'] = df_vacant_2021.number + ' ' +  df_vacant_2021.address + ", Jersey City, NJ"

# geocoding

In [5]:
# https://peterhaas-me.medium.com/how-to-geocode-with-python-and-pandas-4cd1d717d3f7

In [6]:
!pip install geopy nominatim

In [7]:
from geopy.geocoders import Nominatim
from geopandas.tools import geocode
geolocator = Nominatim(timeout=10, user_agent = "jc-apra")

### 2021 abandoned

In [9]:
df_abandoned_2021['gcode'] = df_abandoned_2021.full_address.apply(geolocator.geocode)
# todo print the ones where 'gcode' is na
not_geocoded = df_abandoned_2021[df_abandoned_2021.isna().any(axis=1)]
not_geocoded

,address,block,lot,full_address,gcode
33,1904 Kennedy Boulevard aka JFK Blv2d5503,9,NaN,"1904 Kennedy Boulevard aka JFK Blv2d5503, Jers...",None
37,10 Morton Place 22405,5,NaN,"10 Morton Place 22405, Jersey City, NJ","(10, Morton Place, West Bergen, Jersey City, H..."


In [10]:
# drop the not geocoded and copy geocoded coods to geometry
df_abandoned_2021.dropna(axis=0, how='any', inplace=True )
df_abandoned_2021['lat'] = [g.latitude for g in df_abandoned_2021.gcode]
df_abandoned_2021['lon'] = [g.longitude for g in df_abandoned_2021.gcode]

In [11]:
### 2021 vacant

In [ ]:
df_vacant_2021['gcode'] = df_vacant_2021.full_address.apply(geolocator.geocode)
# todo print the ones where 'gcode' is na
not_geocoded = df_vacant_2021[df_vacant_2021.isna().any(axis=1)]
not_geocoded

In [ ]:
# drop the not geocoded and copy geocoded coods to geometry
df_vacant_2021.dropna(axis=0, how='any', inplace=True )
df_vacant_2021['lat'] = [g.latitude for g in df_vacant_2021.gcode]
df_vacant_2021['lon'] = [g.longitude for g in df_vacant_2021.gcode]

# mapping

In [ ]:
# ! pip install folium geopandas

In [ ]:
import folium, geopandas

## map 0 standard markers

In [ ]:
import folium

In [ ]:
my_map = folium.Map(
    location=[40.7128,-74.1],
    tiles='Stamen Toner',
    zoom_start=14)


for index, row in df_vacant_2021.iterrows():
    folium.Marker(location=(row['lat'],row['lon']), popup=str(row.number + row.address), icon=folium.Icon(color='orange',icon='home', icon_color='#FFffff')).add_to(my_map) 
    
for index, row in df_abandoned_2021.iterrows():
    folium.Marker(location=(row['lat'],row['lon']), popup=str(row.address), icon=folium.Icon(color='red',icon='home', icon_color='#FFffff')).add_to(my_map)

    
display(my_map)

In [ ]:
# map2 highlighting parcel polygons

In [ ]:
# convert the dataframes to geodataframes
# Convert the DataFrame's content (e.g. Lat and Lon columns) into appropriate Shapely geometries first and then use them together with the original DataFrame to create a GeoDataFrame.

from geopandas import GeoDataFrame
from shapely.geometry import Point


# abandoned
geometry_abandoned = [Point(xy) for xy in zip(df_abandoned_2021.lon, df_abandoned_2021.lat)]
df_abandoned_2021 = df_abandoned_2021.drop(['lon', 'lat'], axis=1)
gdf_abandoned_2021 = GeoDataFrame(df_abandoned_2021, crs="EPSG:4326", geometry=geometry_abandoned)

# vacants
geometry_vacant = [Point(xy) for xy in zip(df_vacant_2021.lon, df_vacant_2021.lat)]
df_vacant_2021 = df_vacant_2021.drop(['lon', 'lat'], axis=1)
gdf_vacant_2021 = GeoDataFrame(df_abandoned_2021, crs="EPSG:4326", geometry=geometry_vacant)



----------------------------------------------------------------------------------------------------

## TODO a map using marker clusters (what's the mapped variable?)

In [19]:
# # folium marker clusters
# from folium.plugins import MarkerCluster
# locations = []

# # City location geometries to a list of latlongs pairs  
# for idx, row in gdf.iterrows():
#     locations.append([row['geometry'].y, row['geometry'].x])

#     # Empty canvas
# m = folium.Map(
#     location=[40.7128,-74.1],
#     tiles='Stamen Toner',
#     zoom_start=12)

# # Markercluster
# m.add_child(MarkerCluster(locations=locations))
# m

## TODO a heatmap

In [20]:
# # folium heat map
# from folium.plugins import HeatMap
# m = folium.Map()
# m.add_child(HeatMap(locations, radius=15))
# m

## TODO a chloropleth on parcels 9what's the mapped variable?)

In [21]:
# chloropleth mapping in Folium
# https://medium.com/analytics-vidhya/create-and-visualize-choropleth-map-with-folium-269d3fd12fa0

In [22]:
# # parcels map
# style_function = lambda x: {'fillColor': '#00ff00'}

# jc_parcels_gdf = geopandas.read_file('https://njgin.state.nj.us/download2/parcels/parcels_shp_dbf_Hudson.zip')
# folium.GeoJson(data=jc_parcels_gdf["geometry"], style_function=style_function).add_to(map)
# map

# 2014 and 2018 data

In [23]:
# 2014 to 2018 csvs and xlsx
# https://data.jerseycitynj.gov/explore/dataset/vacant-building-inventory-and-abandoned-properties-rehabilitation-act/information/

import requests

vacant_buildings_and_lots_2014 = 'https://us.ftp.opendatasoft.com/analyzejerseycity/files/Vacant%20Buildings%20and%20Lots/vacantbuildingslotslist2014csv.csv'
vacant_buildings_2018 = 'https://us.ftp.opendatasoft.com/analyzejerseycity/files/Vacant%20Buildings%20and%20Lots/vacantbuildinginventory2018csv.csv'
abandoned_2016_07 = 'https://us.ftp.opendatasoft.com/analyzejerseycity/files/Vacant%20Buildings%20and%20APRA%20properties/apraproperties201607.xlsx'
abandoned_2017_01 = 'https://us.ftp.opendatasoft.com/analyzejerseycity/files/Vacant%20Buildings%20and%20APRA%20properties/apraproperties201701.xlsx'
abandoned_2017 = 'https://us.ftp.opendatasoft.com/analyzejerseycity/files/Vacant%20Buildings%20and%20APRA%20properties/publicofficersapralist2018.xlsx'

def download_file(url):
    import requests
    response = requests.get(url)
    with open('temp.pdf', 'wb') as f:
        f.write(response.content)
    return 'temp.pdf'

# df = pd.read_csv(download_file(abandoned_2021_07)